In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error,
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
import joblib
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../diamonds_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53794 entries, 0 to 53793
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53794 non-null  float64
 1   cut      53794 non-null  object 
 2   color    53794 non-null  object 
 3   clarity  53794 non-null  object 
 4   depth    53794 non-null  float64
 5   table    53794 non-null  float64
 6   price    53794 non-null  float64
 7   x        53794 non-null  float64
 8   y        53794 non-null  float64
 9   z        53794 non-null  float64
dtypes: float64(7), object(3)
memory usage: 4.1+ MB


In [3]:
df.head()

carat      cut color clarity  depth  table  price     x     y     z
0   0.23    Ideal     E     SI2   61.5   55.0  326.0  3.95  3.98  2.43
1   0.21  Premium     E     SI1   59.8   61.0  326.0  3.89  3.84  2.31
2   0.23     Good     E     VS1   56.9   65.0  327.0  4.05  4.07  2.31
3   0.29  Premium     I     VS2   62.4   58.0  334.0  4.20  4.23  2.63
4   0.31     Good     J     SI2   63.3   58.0  335.0  4.34  4.35  2.75

In [4]:
X = df.drop('cut', axis=1)
y = df['cut']

In [5]:
# Creamos y ajustamos el LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
orden_clases = label_encoder.classes_

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Pasamos LabelEncoder a la y_test y la y_train
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [8]:
# Categorías y Columnas Numéricas
categorical_cols = ["color", "clarity"]  # Hemos Excluido 'cut' al dividir 
numerical_cols = ["carat", "depth", "table", "price", "x", "y", "z"]

In [9]:
# Crear pipelines de preprocesamiento
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
# Combinar pipelines en un ColumnTransformer
preprocessor = ColumnTransformer([
    ('numerical', numerical_pipeline, numerical_cols),
    ('categorical', categorical_pipeline, categorical_cols)
])

In [10]:
# Crear el pipeline del modelo
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [11]:
# Entrenar el modelo
model.fit(X_train, y_train_encoded)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['carat', 'depth', 'table',
                                                   'price', 'x', 'y', 'z']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['color', 'clarity'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [12]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
# Evaluar el modelo
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test_encoded, y_pred))
print("F1-score:", f1_score(y_test_encoded, y_pred, average='weighted'))
print("Recall:", recall_score(y_test_encoded, y_pred, average='weighted'))
print("Precision:", precision_score(y_test_encoded, y_pred, average='weighted'))

Accuracy: 0.766985779347523
F1-score: 0.7589800376197546
Recall: 0.766985779347523
Precision: 0.7601576118230176


In [13]:
# Guardar el modelo y el LabelEncoder
joblib.dump(model, '../models/pipeline_clasificacion.joblib')
joblib.dump(label_encoder, '../models/label_encoder_cut.joblib')
print("Modelo y LabelEncoder guardados exitosamente.")

Modelo y LabelEncoder guardados exitosamente.
